In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import math
import tensorflow.python.keras as keras
from keras.layers import Input, Layer, LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from keras.models import Model
import keras.backend as K
import os
print("Running Tensorflow version", tf.__version__)

Running Tensorflow version 2.1.0


Using TensorFlow backend.


In [2]:
class Round(Layer):
    def __init__(self, **kwargs):
        super(Round, self).__init__(**kwargs)
    def get_output(self, train=False):
        X = self.get_input(train)
        return K.clip(K.round(X * 4096), -524288, 524287) / 4096.0
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(Round, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
class Integer(Layer):
    def __init__(self, **kwargs):
        super(Integer, self).__init__(**kwargs)
    def get_output(self, train=False):
        X = self.get_input(train)
        return K.clip(K.round(X * 4096), -524288, 524287) / 16777216.0
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(Integer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
class Identity(Layer):
    def __init__(self, **kwargs):
        super(Identity, self).__init__(**kwargs)
    def get_output(self, train=False):
        X = self.get_input(train)
        return X
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(Identity, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
def DBL(previousLayer, layerFilter, kernelSize=(3, 3), roundingFunction=Identity()):
    return roundingFunction(LeakyReLU()(roundingFunction(BatchNormalization()(Conv2D(filters=layerFilter, kernel_size=kernelSize, padding='same')(previousLayer)))))
print("Custom layer classes successfully defined")

Custom layer classes successfully defined


In [3]:
classificationClass = 4
print("Number of class classification is", classificationClass)

Number of class classification is 4


In [4]:
# model_0 does no rounding (float32 operation)
model_0_input = Input(shape=(448, 448, 3))
model_0_pointer = model_0_input
print("Input shape:", model_0_pointer.shape) # 448 x 448 x 3
model_0_startBranch = DBL(previousLayer=model_0_input, layerFilter=16) 
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 448 x 448 x 16
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 224 x 224 x 16
model_0_startBranch = DBL(previousLayer=model_0_startBranch, layerFilter=32)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 224 x 224 x 32
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 112 x 112 x 32
model_0_startBranch = DBL(previousLayer=model_0_startBranch, layerFilter=64)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 112 x 112 x 64
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 56 x 56 x 64
model_0_startBranch = DBL(previousLayer=model_0_startBranch, layerFilter=128)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 56 x 56 x 128
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 28 x 28 x 128
model_0_startBranch = DBL(previousLayer=model_0_startBranch, layerFilter=256)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_0_branch0
model_0_branch0 = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 256
model_0_branch0 = DBL(previousLayer=model_0_branch0, layerFilter=512)
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 512
model_0_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_0_branch0)
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 512
model_0_branch0 = DBL(previousLayer=model_0_branch0, layerFilter=1024)
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 1024
model_0_branch0 = DBL(previousLayer=model_0_branch0, layerFilter=256, kernelSize=(1, 1))
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_0_branch0 (14 x 14 x 256), following model_0_branch00
model_0_branch00 = DBL(previousLayer=model_0_branch0, layerFilter=128, kernelSize=(1, 1))
model_0_pointer = model_0_branch00
print(model_0_pointer.shape) # 14 x 14 x 128
model_0_branch00 = UpSampling2D()(model_0_branch00)
model_0_pointer = model_0_branch00
print(model_0_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_0_branch1 (unchanged from model_0_startBranch) and model_0_branch00
model_0_mergedBranch = Concatenate()([model_0_startBranch, model_0_branch00])
model_0_pointer = model_0_mergedBranch
print(model_0_pointer.shape) # 28 x 28 x 384
model_0_mergedBranch = DBL(previousLayer=model_0_mergedBranch, layerFilter=256)
model_0_pointer = model_0_mergedBranch
print(model_0_pointer.shape) # 28 x 28 x 256
model_0_mergedBranch = DBL(previousLayer=model_0_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass))
model_0_pointer = model_0_mergedBranch
print("Model output 0 shape:", model_0_pointer.shape) # 28 x 28 x (3 * (5 + classificationClass))
print() # OUTPUT = model_0_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1")# following model_0_branch01
model_0_branch01 = DBL(previousLayer=model_0_branch0, layerFilter=512)
model_0_pointer = model_0_branch01
print(model_0_pointer.shape) # 14 x 14 x 512
model_0_branch01 = DBL(previousLayer=model_0_branch01, layerFilter=3 * (4 + 1 + classificationClass))
model_0_pointer = model_0_branch01
print("Model output 1 shape:", model_0_pointer.shape) # 14 x 14 x (3 * (5 + classificationClass))
print() # OUTPUT = model_0_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_0 = Model(inputs=model_0_input, outputs=[model_0_mergedBranch, model_0_branch01])
model_0.compile(optimizer='rmsprop', loss='binary_crossentropy', loss_weights=[1., 0.2])
print("Model model_0 compilation complete")

Input shape: (None, 448, 448, 3)
(None, 448, 448, 16)
(None, 224, 224, 16)
(None, 224, 224, 32)
(None, 112, 112, 32)
(None, 112, 112, 64)
(None, 56, 56, 64)
(None, 56, 56, 128)
(None, 28, 28, 128)
(None, 28, 28, 256)
Branch split from main branch - following branch 0
(None, 14, 14, 256)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 1024)
(None, 14, 14, 256)
Branch split from branch 0 - following branch 0,0
(None, 14, 14, 128)
(None, 28, 28, 128)
Branch merge from branch 1 and branch 0,0
(None, 28, 28, 384)
(None, 28, 28, 256)
Model output 0 shape: (None, 28, 28, 27)

Branch split from branch 0 - following branch 0,1
(None, 14, 14, 512)
Model output 1 shape: (None, 14, 14, 27)

Model model_0 compilation complete


In [5]:
# model_1 approximates Q7.12 signed fixed point operations 
# Done by rounding to the nearest 1/4096 and capping at [-128, 128) after batch normalization and activation layers
model_1_input = Input(shape=(448, 448, 3))
model_1_pointer = model_1_input
print("Input shape:", model_1_pointer.shape) # 448 x 448 x 3
model_1_startBranch = DBL(roundingFunction=Round(), previousLayer=model_1_input, layerFilter=16) 
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 448 x 448 x 16
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 224 x 224 x 16
model_1_startBranch = DBL(roundingFunction=Round(), previousLayer=model_1_startBranch, layerFilter=32)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 224 x 224 x 32
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 112 x 112 x 32
model_1_startBranch = DBL(roundingFunction=Round(), previousLayer=model_1_startBranch, layerFilter=64)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 112 x 112 x 64
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 56 x 56 x 64
model_1_startBranch = DBL(roundingFunction=Round(), previousLayer=model_1_startBranch, layerFilter=128)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 56 x 56 x 128
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 28 x 28 x 128
model_1_startBranch = DBL(roundingFunction=Round(), previousLayer=model_1_startBranch, layerFilter=256)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_1_branch0
model_1_branch0 = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 256
model_1_branch0 = DBL(roundingFunction=Round(), previousLayer=model_1_branch0, layerFilter=512)
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 512
model_1_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_1_branch0)
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 512
model_1_branch0 = DBL(roundingFunction=Round(), previousLayer=model_1_branch0, layerFilter=1024)
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 1024
model_1_branch0 = DBL(roundingFunction=Round(), previousLayer=model_1_branch0, layerFilter=256, kernelSize=(1, 1))
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_1_branch0 (14 x 14 x 256), following model_1_branch00
model_1_branch00 = DBL(roundingFunction=Round(), previousLayer=model_1_branch0, layerFilter=128, kernelSize=(1, 1))
model_1_pointer = model_1_branch00
print(model_1_pointer.shape) # 14 x 14 x 128
model_1_branch00 = UpSampling2D()(model_1_branch00)
model_1_pointer = model_1_branch00
print(model_1_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_1_branch1 (unchanged from model_1_startBranch) and model_1_branch00
model_1_mergedBranch = Concatenate()([model_1_startBranch, model_1_branch00])
model_1_pointer = model_1_mergedBranch
print(model_1_pointer.shape) # 28 x 28 x 384
model_1_mergedBranch = DBL(roundingFunction=Round(), previousLayer=model_1_mergedBranch, layerFilter=256)
model_1_pointer = model_1_mergedBranch
print(model_1_pointer.shape) # 28 x 28 x 256
model_1_mergedBranch = DBL(roundingFunction=Round(), previousLayer=model_1_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass))
model_1_pointer = model_1_mergedBranch
print("Model output 0 shape:", model_1_pointer.shape) # 28 x 28 x (3 * (5 + classificationClass))
print() # OUTPUT = model_1_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1") # following model_1_branch01
model_1_branch01 = DBL(roundingFunction=Round(), previousLayer=model_1_branch0, layerFilter=512)
model_1_pointer = model_1_branch01
print(model_1_pointer.shape) # 14 x 14 x 512
model_1_branch01 = DBL(roundingFunction=Round(), previousLayer=model_1_branch01, layerFilter=3 * (4 + 1 + classificationClass))
model_1_pointer = model_1_branch01
print("Model output 1 shape:", model_1_pointer.shape) # 14 x 14 x (3 * (5 + classificationClass))
print() # OUTPUT = model_1_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_1 = Model(inputs=model_1_input, outputs=[model_1_mergedBranch, model_1_branch01])
model_1.compile(optimizer='rmsprop', loss='binary_crossentropy', loss_weights=[1., 0.2])
print("Model model_1 compilation complete")

Input shape: (None, 448, 448, 3)
(None, 448, 448, 16)
(None, 224, 224, 16)
(None, 224, 224, 32)
(None, 112, 112, 32)
(None, 112, 112, 64)
(None, 56, 56, 64)
(None, 56, 56, 128)
(None, 28, 28, 128)
(None, 28, 28, 256)
Branch split from main branch - following branch 0
(None, 14, 14, 256)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 1024)
(None, 14, 14, 256)
Branch split from branch 0 - following branch 0,0
(None, 14, 14, 128)
(None, 28, 28, 128)
Branch merge from branch 1 and branch 0,0
(None, 28, 28, 384)
(None, 28, 28, 256)
Model output 0 shape: (None, 28, 28, 27)

Branch split from branch 0 - following branch 0,1
(None, 14, 14, 512)
Model output 1 shape: (None, 14, 14, 27)

Model model_1 compilation complete


In [6]:
# model_2 approximates integer operations by multiplying result by 4096 -> rounding -> dividing by 16777216, on attempt to multiply the kernel weight values by 4096, therefore closer to integer
model_2_input = Input(shape=(448, 448, 3))
model_2_pointer = model_2_input
print("Input shape:", model_2_pointer.shape) # 448 x 448 x 3
model_2_startBranch = DBL(roundingFunction=Integer(), previousLayer=model_2_input, layerFilter=16) 
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 448 x 448 x 16
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 224 x 224 x 16
model_2_startBranch = DBL(roundingFunction=Integer(), previousLayer=model_2_startBranch, layerFilter=32)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 224 x 224 x 32
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 112 x 112 x 32
model_2_startBranch = DBL(roundingFunction=Integer(), previousLayer=model_2_startBranch, layerFilter=64)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 112 x 112 x 64
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 56 x 56 x 64
model_2_startBranch = DBL(roundingFunction=Integer(), previousLayer=model_2_startBranch, layerFilter=128)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 56 x 56 x 128
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 28 x 28 x 128
model_2_startBranch = DBL(roundingFunction=Integer(), previousLayer=model_2_startBranch, layerFilter=256)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_2_branch0
model_2_branch0 = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 256
model_2_branch0 = DBL(roundingFunction=Integer(), previousLayer=model_2_branch0, layerFilter=512)
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 512
model_2_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_2_branch0)
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 512
model_2_branch0 = DBL(roundingFunction=Integer(), previousLayer=model_2_branch0, layerFilter=1024)
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 1024
model_2_branch0 = DBL(roundingFunction=Integer(), previousLayer=model_2_branch0, layerFilter=256, kernelSize=(1, 1))
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_2_branch0 (14 x 14 x 256), following model_2_branch00
model_2_branch00 = DBL(roundingFunction=Integer(), previousLayer=model_2_branch0, layerFilter=128, kernelSize=(1, 1))
model_2_pointer = model_2_branch00
print(model_2_pointer.shape) # 14 x 14 x 128
model_2_branch00 = UpSampling2D()(model_2_branch00)
model_2_pointer = model_2_branch00
print(model_2_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_2_branch1 (unchanged from model_2_startBranch) and model_2_branch00
model_2_mergedBranch = Concatenate()([model_2_startBranch, model_2_branch00])
model_2_pointer = model_2_mergedBranch
print(model_2_pointer.shape) # 28 x 28 x 384
model_2_mergedBranch = DBL(roundingFunction=Integer(), previousLayer=model_2_mergedBranch, layerFilter=256)
model_2_pointer = model_2_mergedBranch
print(model_2_pointer.shape) # 28 x 28 x 256
model_2_mergedBranch = DBL(roundingFunction=Integer(), previousLayer=model_2_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass))
model_2_pointer = model_2_mergedBranch
print("Model output 0 shape:", model_2_pointer.shape) # 28 x 28 x (3 * (5 + classificationClass))
print() # OUTPUT = model_2_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1")# following model_2_branch01
model_2_branch01 = DBL(roundingFunction=Integer(), previousLayer=model_2_branch0, layerFilter=512)
model_2_pointer = model_2_branch01
print(model_2_pointer.shape) # 14 x 14 x 512
model_2_branch01 = DBL(roundingFunction=Integer(), previousLayer=model_2_branch01, layerFilter=3 * (4 + 1 + classificationClass))
model_2_pointer = model_2_branch01
print("Model output 1 shape:", model_2_pointer.shape) # 14 x 14 x (3 * (5 + classificationClass))
print() # OUTPUT = model_2_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_2 = Model(inputs=model_2_input, outputs=[model_2_mergedBranch, model_2_branch01])
model_2.compile(optimizer='rmsprop', loss='binary_crossentropy', loss_weights=[1., 0.2])
print("Model model_2 compilation complete")

Input shape: (None, 448, 448, 3)
(None, 448, 448, 16)
(None, 224, 224, 16)
(None, 224, 224, 32)
(None, 112, 112, 32)
(None, 112, 112, 64)
(None, 56, 56, 64)
(None, 56, 56, 128)
(None, 28, 28, 128)
(None, 28, 28, 256)
Branch split from main branch - following branch 0
(None, 14, 14, 256)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 1024)
(None, 14, 14, 256)
Branch split from branch 0 - following branch 0,0
(None, 14, 14, 128)
(None, 28, 28, 128)
Branch merge from branch 1 and branch 0,0
(None, 28, 28, 384)
(None, 28, 28, 256)
Model output 0 shape: (None, 28, 28, 27)

Branch split from branch 0 - following branch 0,1
(None, 14, 14, 512)
Model output 1 shape: (None, 14, 14, 27)

Model model_2 compilation complete


In [7]:
print(model_0.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 448, 448, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 448, 448, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 448, 448, 16) 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
identity_1 (Identity)           multiple             0           batch_normalization_1[0][0]      
                                                                 leaky_re_lu_1[0][0]        

In [8]:
print(model_1.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 448, 448, 3)  0                                            
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 448, 448, 16) 448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_14 (BatchNo (None, 448, 448, 16) 64          conv2d_14[0][0]                  
__________________________________________________________________________________________________
round_1 (Round)                 (None, 448, 448, 16) 0           batch_normalization_14[0][0]     
                                                                 leaky_re_lu_14[0][0]       

In [9]:
print(model_2.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 448, 448, 3)  0                                            
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, 448, 448, 16) 448         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_27 (BatchNo (None, 448, 448, 16) 64          conv2d_27[0][0]                  
__________________________________________________________________________________________________
integer_1 (Integer)             (None, 448, 448, 16) 0           batch_normalization_27[0][0]     
                                                                 leaky_re_lu_27[0][0]       